# Nginx

- https://nginx.org
- https://hub.docker.com/_/nginx

## Ports

<img src=images/docker-drillingholes.jpg height=400>

## Volumes

<img src=images/docker-volumes.png height=400>

In [ ]:
docker run -p 5000:80 -v .\3-nginx:/usr/share/nginx/html/ nginx

# PostgreSQL

- https://www.postgresql.org
- https://hub.docker.com/_/postgres

## Environment Variables

<img src=images/docker-environment-variables.jpg height=400>

## Data Analysis

- https://www.kaggle.com/datasets/carrie1/ecommerce-data?resource=download
- https://github.com/atanaskanev/ecommerce_sales_analysis

# Homework

In [ ]:
import os
import pandas as pd

# Read dataset into pandas
df = pd.read_csv('grafana/data_import/data.csv', encoding = 'unicode_escape')
df.InvoiceNo = pd.to_numeric(df.InvoiceNo, errors='coerce')

df['InvoiceNo'][0]

In [ ]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    database=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD')
)

# Define SQL query with parameters
sql = """
    INSERT INTO sales_data (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

cursor = conn.cursor()

# Iterate over rows in DataFrame and execute parameterized query
for index, row in df.iterrows():
    cursor.execute(sql, (
        int(row['InvoiceNo']),
        row['StockCode'],
        row['Description'],
        int(row['Quantity']),
        pd.to_datetime(row['InvoiceDate']).tz_localize('UTC'),  # Ensure UTC timezone
        float(row['UnitPrice']),
        int(row['CustomerID']) if not pd.isnull(row['CustomerID']) else None,
        row['Country']
    ))

conn.commit()
cursor.close()
conn.close()